<a href="https://colab.research.google.com/github/kkkkkkkm/Torch/blob/main/RNN/word_RNN_using_Embedding_not_Yet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import torch
import torch.nn
import torch.optim as optim

In [23]:
sentence = "Repeat is the best medicien for memory".split() #띄어 쓰기를 기준으로 분리

In [24]:
print(sentence) #분리된 상황

['Repeat', 'is', 'the', 'best', 'medicien', 'for', 'memory']


In [25]:
vocab = list(set(sentence)) #중복 제거 후 list에 담음
print(vocab) #vocab(단어장)의 사이즈는 label의 크기를 나타냄 

['is', 'best', 'Repeat', 'the', 'memory', 'medicien', 'for']


In [26]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)} #enumerate start = 1
word2index['<unk>'] = 0 #단어 사전에 unknown = 0 삽입

In [27]:
print(word2index) # (str : 정수) 타입 dict

{'is': 1, 'best': 2, 'Repeat': 3, 'the': 4, 'memory': 5, 'medicien': 6, 'for': 7, '<unk>': 0}


In [28]:
print(word2index['memory'])

5


In [29]:
word2index.items()

dict_items([('is', 1), ('best', 2), ('Repeat', 3), ('the', 4), ('memory', 5), ('medicien', 6), ('for', 7), ('<unk>', 0)])

In [30]:
index2word = {v:k for k ,v in word2index.items()} #word2index dictionary에 있는 값들을 전달해줌
print(index2word)

{1: 'is', 2: 'best', 3: 'Repeat', 4: 'the', 5: 'memory', 6: 'medicien', 7: 'for', 0: '<unk>'}


In [31]:
print(index2word[2])

best


In [45]:
def build_data(sentence, word2index):
  encoded = [word2index[token] for token in sentence] #각 문자를 정수로 변환
  input_seq, label_seq = encoded[:-1], encoded[1:] #현재 list형태 
  input_seq = torch.LongTensor(input_seq).unsqueeze(0) #list를 tensor로 변경 후 현재 1차원 벡터인 input_seq을 unsqueeze(0)을 통해 배치가 1인 2차원 벡터로 변경
  label_seq = torch.LongTensor(label_seq).unsqueeze(0) #위와 같음
  
  return input_seq, label_seq

In [46]:
X, Y = build_data(sentence, word2index)

In [47]:
print(X)
print(Y)

tensor([[3, 1, 4, 2, 6, 7]])
tensor([[1, 4, 2, 6, 7, 5]])


In [ ]:
class Net(nn.Module):
  
  def __init__(self, vocab_size, input_size ,hidden_size, batch_first = True):
    super(Net, self).__init__()
    self.embedding_layer = nn.Embedding(num_embedding = vocab_size, embedding_dim = input_size) #num_embedding은 단어장의 크기가 들어오고(입력차원의 크기), embedding_dim은 embedding 벡터의 차원(출력차원)

    self.rnn_layer = nn.RNN(input_size, hidden_size, batch_first = True) #입력 차원 크기 = input_size, 출력차원의 크기 = hidden_size
    self.linear = nn.Linear(hidden_size, vocab_size) #입력은 hidden_size 만큼 받아서 vocab_size로 출력, 후에 정답을 맞추기 쉽게

 def forward(self, x):
   # 1. 임베딩 층
   # 크기 변화 : (배치, 시퀀스 길이) => 배치, 시퀀스 , 임베딩 차원
   output = self.embedding_layer(x)
  # 2. RNN 층
  # 크기변화 (배치, 시퀀스, 임베딩 차원)
  # => output = (배치크기, 시퀀스 길이, 은닉층 크기), hidden(마지막 시점의 은닉상태) = (1, 배치크기, 은닉층 크기) 1은 층의 숫자를 의미함
   output, hidden = self.rnn_layer(output)

   output = self.linear(output)